<a href="https://colab.research.google.com/github/kevgam/CAS_IE_Information_Retrieval/blob/main/SpotifySongClassifier_Gesamt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitungsarbeiten in Spark

# Vorbereitung des Datensatzes in Spark

Unser ursprünglicher Spotify-Datensatz, der über [Kaggle](https://www.kaggle.com/discussions/accomplishments/522912) verfügbar ist, umfasste fast eine Million Datensätze. Aufgrund der Grösse des Datensatzes hatten wir bei der Verarbeitung Performanceprobleme. Daher haben wir den Datensatz zunächst in der Spark-Umgebung der ZHAW vorverarbeitet.


Nach dem Standardlogin (inkl. sc.stop() am Schluss) gemäss Anleitung der ZHAW haben wir folgende Schritte ausgeführt:

### Installation und Test der Umgebung

```python
# Installation des notwendigen Pakets
sparky.installpackage('langdetect')

# Alternativ mit pip
pip install langdetect

# Test der RDD-Funktionalität
import os
liste = range(16)
rdd = sc.parallelize(liste)
print(rdd.collect())
print(rdd.glom().collect())

# Überprüfen, ob alle Worker die notwendige Software installiert haben
if len(list(filter(lambda x: x == [], rdd.glom().collect()))):
    raise SystemExit("Nicht gut - einige Worker bleiben ohne Softwareinstallation.")

# Testfunktion für Abhängigkeiten
def testdep(ignore_arg):
    ip = "160.85.252.66"  # Beispiel-IP
    try:
        import lxml
    except:
        return f"lxml FAILED! @ {ip}"
    else:
        return f"lxml worked @ {ip}"

# Installation von Abhängigkeiten
import subprocess
def installdeps(ignore_arg):
    p = subprocess.run("pip install lxml", shell=True, stdout=subprocess.PIPE)
    return p.stdout.decode()

# Ausführen der Installation und Tests
rdd.map(installdeps).collect()
rdd.map(testdep).collect()


### Laden und Verarbeiten der Daten

```python
# Das File wurde vorgängig in unseren Ordner auf dem Server kopiert
filepath = 'songs_with_attributes_and_lyrics.csv'

# Laden der CSV-Datei
import pandas as pd
dfs = pd.read_csv(filepath)

# Spark DataFrame laden
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
dfs = spark.read.csv(filepath, header=True, inferSchema=True)


### Sprache erkennen und Fortschritt protokollieren

```python
from langdetect import detect

# Funktion zur Erkennung der Sprache mit Fortschrittsanzeige
def detect_language_with_progress(partition):
    total_rows = 0
    for row in partition:
        try:
            lang = detect(row['lyrics'])
            yield (row['lyrics'], lang)  # Rückgabe: Originaltext und erkannte Sprache
        except Exception:
            yield (row['lyrics'], 'unknown')
        total_rows += 1
        if total_rows % 1000 == 0:  # Fortschritt alle 1000 Zeilen anzeigen
            print(f"Processed {total_rows} rows in this partition")

# RDD-Transformationen anwenden
rdd = dfs.rdd.mapPartitions(detect_language_with_progress)

# Zurück in ein DataFrame umwandeln
schema = StringType()
result = rdd.toDF(["lyrics", "lyrics_language"])

# Fortschritt anzeigen
result.show()


### Ergebnisse speichern

```python
# Als Excel-Datei speichern
output_path = './processed_songs_with_lyrics.xlsx'
dfs.to_excel(output_path, index=False)
print(f"DataFrame saved to: {output_path}")

# Als CSV-Datei speichern
csv_output_path = './processed_songs_with_lyrics.csv'
dfs.to_csv(csv_output_path, index=False)
print(f"DataFrame saved as CSV file to: {csv_output_path}")


### Filterung und Speicherung der englischen Texte

```python
# Zeilen filtern, in denen die Sprache Englisch ist
dfs_en = dfs[dfs['lyrics_language'] == 'en']

# Gefilterte Daten als Excel- und CSV-Datei speichern
dfs_en.to_excel('./processed_songs_filtered_lyrics_en.xlsx', index=False)
dfs_en.to_csv('./processed_songs_filtered_lyrics_en.csv', index=False)

print("Filtered DataFrame saved as 'filtered_lyrics_en.xlsx' and 'filtered_lyrics_en.csv'")


# Aufbereitung der Daten für Spotify Song Classifer

## Google Drive verbinden

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Installation der Python-Pakete

In [2]:
# Installation
!pip install pandas openpyxl

## Import der Bibliotheken

In [3]:
# Für Datenbereinigung
import pandas as pd

# Für das Parsen und Analysieren
import ast

# Für das Zählen von Elementen
from collections import Counter

# Für die Erstellung von Visualisierungen
import matplotlib.pyplot as plt
import seaborn as sns

# Für natürliche Sprachverarbeitung (Natural Language Processing)
import nltk

# Importiere die 'stopwords'-Liste und die Funktion um Text in einzelne Wörter zu zerlegen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Für die Tokenisierung von Texten und Entfernung von Stopwörtern
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Für reguläre Ausdrücke für die Textverarbeitung
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Laden der Datei, Filtern und Datenbereinigungen

In [ ]:
# Laden der Excel-Datei
file_path = "/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_en.xlsx"
df = pd.read_excel(file_path)

# Entfernung der Spalten 'id' und 'album_name'
df = df.drop(columns=['id', 'album_name', 'lyrics_language'], errors='ignore')

# Entfernung aller Datensätze, bei denen 'duration_ms' kleiner als 240000 (4 Minuten) oder größer als 300000 (5 Minuten) ist
df = df[(df['duration_ms'] >= 240000) & (df['duration_ms'] <= 300000)]

# Entfernung der Sonderzeichen in den Spalten 'name' und 'artists'
# Sonderzeichen inkl. [] werden entfernt
df['name'] = df['name'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['artists'] = df['artists'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# Filterung aller Zeilen, bei denen 'name' NaN ist
df = df.dropna(subset=['name'])

# Filterung aller Zeilen, bei denen 'artists' NaN ist
df = df.dropna(subset=['artists'])

# Entfernung Duplikate basierend auf der Kombination aus 'name' und 'artists'
df = df.drop_duplicates(subset=['name', 'artists'])

# Sicherstellen, dass die Spalte 'lyrics' nur String-Werte enthält
df = df[df['lyrics'].apply(lambda x: isinstance(x, str))]

# Entfernung der Zeilenumbrüche innerhalb der 'lyrics' Spalte
df['lyrics'] = df['lyrics'].apply(lambda x: str(x).replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)

# Anführungszeichen zur Texterkennung in der 'lyrics'-Spalte
df['lyrics'] = df['lyrics'].apply(lambda x: f'"{x}"' if isinstance(x, str) else x)

# Entfernung der Duplikate basierend auf der Kombination aus 'duration_ms' und 'lyrics'
df = df.drop_duplicates(subset=['duration_ms', 'lyrics'])

# Speicherung der bereinigten Daten als CSV-Datei
output_csv_path = '/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv'
df.to_csv(output_csv_path, index=False)

print(f"Bereinigte CSV-Datei wurde gespeichert unter: {output_csv_path}")


ERROR: Operation cancelled by user


KeyboardInterrupt: 

## Bereinigung der Songtexte (Tokenisierung, Kleinschreibung, Entfernung Sonderzeichen und Stopwörter)

In [ ]:
# Laden des Dataframes
df = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv')

# Sicherstellen, dass die Spalte 'lyrics' keine fehlenden Werte enthält und Strings sind
df['lyrics'] = df['lyrics'].fillna('').astype(str)

# Funktion zur Textbereinigung
def preprocess_text(text):
    if not isinstance(text, str):  # Sicherstellen, dass die Eingabe ein String ist
        text = str(text)
    tokens = word_tokenize(text.lower())  # Tokenisierung & Kleinschreibung
    tokens = [word for word in tokens if word.isalpha()]  # Sonderzeichen/Zahlen entfernen
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopwörter entfernen
    return tokens  # Rückgabe als Liste von Tokens

# Bereinigte Tokens in neuer Spalte speichern
df['clean_lyrics'] = df['lyrics'].apply(preprocess_text)

# Ergebnisse speichern
df.to_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_without_stopwords.csv', index=False)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Entfernung Duplikate und NaN-Werte in Lyrics

In [ ]:
# Laden des Dataframes
df = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_without_stopwords.csv')

# Entferne Duplikate in 'clean_lyrics'
df = df.drop_duplicates(['clean_lyrics'])

# Filtere alle Zeilen, bei denen 'clean_lyrics' NaN ist
df = df.dropna(subset=['clean_lyrics'])

# Ergebnisse speichern
df.to_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_without_duplicates.csv', index=False)


## Zuordnung der Emotionen auf Basis der Songtexte mittels NRC Emotion Lexikon und der Spotify-Metadaten


### Das NRC Emotion Lexikon ordnet Wörter bestimmten Emotionen (z. B. Freude, Traurigkeit, Angst) sowie positiven oder negativen Stimmungen zu.

### Das Lexikon ist unter folgendem Link verfügbar:
https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm

In [ ]:
# Daten laden
data = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_without_duplicates.csv')


# Lade das NRC Emotion Lexikon
nrc_lexicon = pd.read_csv(
    '/content/drive/MyDrive/ie_scripting_datasets/Archive/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt',
    sep='\t',
    header=None,
    names=['word', 'emotion', 'value']
)


# Nur relevante Einträge aus dem NRC Emotion Lexikon behalten
nrc_lexicon = nrc_lexicon[nrc_lexicon['value'] == 1]

emotion_categories = {
    'anger': ['anger'],
    'fear': ['fear'],
    'anticipation': ['anticipation'],
    'trust': ['trust'],
    'surprise': ['surprise'],
    'sadness': ['sadness'],
    'joy': ['joy'],
    'disgust': ['disgust']
}


# Emotionen aus den Songtexten extrahieren
def get_emotions_and_category(tokens, lexicon, emotion_categories):

    # Überprüfung, ob die Eingabe ein String ist und Umwandlung in eine Liste von Tokens
    if isinstance(tokens, str):
        tokens = ast.literal_eval(tokens)

    # Filterung der Wörter im Lexikon, die in den Tokens vorkommen und Extraktion der zugehörenden Emotionen
    emotions = lexicon[lexicon['word'].isin(tokens)]['emotion'].values

    # Zählung Häufigkeit jeder Emotion
    emotion_counts = Counter(emotions)

    # Summe je Emotion-Kategorie
    category_counts = {
        category: sum(emotion_counts[emotion] for emotion in emotions_list)
        for category, emotions_list in emotion_categories.items()
    }

    # Bestimmung, ob die Stimmung positiv oder negativ ist, basierend auf den 'joy'- und 'sadness'-Werten
    is_positive = category_counts.get('joy', 0) > category_counts.get('sadness', 0)
    sentiment = 'positive' if is_positive else 'negative'

    # Resultate
    return emotion_counts, category_counts, sentiment


# Anwendung der Funktion auf die Spalte 'clean_lyrics'
# Berechnung der Emotions- und Kategoriewerte sowie des Sentiments je Songtext
results = data['clean_lyrics'].apply(
    lambda x: get_emotions_and_category(x, nrc_lexicon, emotion_categories)
)


# Ergänzung der Resultate in die Tabelle:
data['emotion_counts'] = results.map(lambda x: x[0])
data['category_counts'] = results.map(lambda x: x[1])
data['primary_emotion'] = results.map(lambda x: max(x[1], key=x[1].get) if x[1] else 'neutral')


# Funktion für die Ableitung der Emotionen aus den Spotify-Metadaten
# Die Metadaten werden verwendet, um Punktzahlen für verschiedene Emotionen zu berechnen.
# Die Emotion mit den höchsten Punkten wird als primäre Emotion ausgewählt.

def determine_emotion_from_spotify_features(row):
    danceability = row['danceability']
    energy = row['energy']
    loudness = row['loudness']
    valence = row['valence']
    tempo = row['tempo']
    acousticness = row['acousticness']

    # Berechnung der Punkte
    emotion_scores = {
        'joy': (valence * 0.4) + (danceability * 0.3) + (tempo * 0.3),
        'sadness': (1 - valence) * 0.5 + (acousticness * 0.5),
        'anger': (energy * 0.6) + (loudness * 0.4),
        'trust': (acousticness * 0.7) + (danceability * 0.3),
        'fear': (1 - energy) * 0.6 + (1 - loudness) * 0.4,
        'surprise': (tempo * 0.5) + (row['liveness'] * 0.5)
    }

    # Resultate
    return max(emotion_scores, key=emotion_scores.get)

# Anwendung der Funktion auf den gesamten Datensatz, neue Spalte mit der primären Emotion
data['primary_emotion_from_features'] = data.apply(determine_emotion_from_spotify_features, axis=1)



# Funktion für die Kombination der Emotionen
# Bei gleicher Emotion wird diese übernommen. Bei unterschiedlichen erfolgt die Zuordnung aufgrund der Gewichtung.
def combine_emotions(lyrics_emotion, features_emotion):
    # Überprüfung, ob die Emotionen aus den Songtexten und den Spotify-Merkmalen übereinstimmen
    if lyrics_emotion == features_emotion:
        return lyrics_emotion
    else:
        # Gewichtung für jede Emotion basierend auf ihrer Bedeutung
        emotion_weights = {
            'joy': 3,
            'sadness': 3,
            'anger': 2,
            'fear': 2,
            'trust': 1,
            'disgust': 1,
            'anticipation': 1,
            'surprise': 1
        }
        # Aufruf der Gewichtung für die Emotionen aus den Songtexten und den Spotify-Merkmalen
        lyrics_weight = emotion_weights.get(lyrics_emotion, 0)
        features_weight = emotion_weights.get(features_emotion, 0)
        # Gibt die Emotion mit der höheren Gewichtung zurück. Bei Gleichstand wird die Emotion aus den Songtexten übernommen.
        return lyrics_emotion if lyrics_weight >= features_weight else features_emotion

# Anwendung der Funkion auf alle Songs, Erstellung der Spalte "final_emotion"
data['final_emotion'] = data.apply(
    lambda row: combine_emotions(row['primary_emotion'], row['primary_emotion_from_features']), axis=1
)


# Speichern der Ergebnisse
output_csv_path = '/content/drive/MyDrive/ie_scripting_datasets/Archive/songs_with_combined_emotions.csv'
data.to_csv(output_csv_path, index=False)
print(f"Ergebnisse gespeichert: {output_csv_path}")



# Visualisierung und Suchfunktion für Spotify Song Classifier / Auch als separates Notebook gespeichert

## Installation der Python-Pakete

In [ ]:
# Installation

!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00


## Import der Bibliotheken

In [ ]:
# Für interaktive Widgets
import ipywidgets as widgets

# Für natürliche Sprachverarbeitung (Natural Language Processing)
import nltk

# Importiere die 'stopwords'-Liste und die Funktion um Text in einzelne Wörter zu zerlegen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Für die Tokenisierung von Texten und Entfernung von Stopwörtern
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


# Für die Anzeige von Grafiken und die Datenmanipulation
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Für reguläre Ausdrücke für die Textverarbeitung
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Google Drive verbinden

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Interaktive Suche mit ipywidgets

In [ ]:
# Laden der CSV-Datei mit den bereinigten Song-Daten
data = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/songs_with_combined_emotions.csv')



# Funktion zur Visualisierung und Anzeige der Songs nach Emotion
def visualize_emotions_and_display_songs(filtered_data, title="Distribution by Emotion"):
    """
    Diese Funktion visualisiert die Verteilung der Emotionen und zeigt die gefilterten Songs an.
    """
    plt.figure(figsize=(8, 5))

    # Definition der Farben je Emotion
    emotion_colors = {
        'anger': '#FF5733', 'fear': '#C70039', 'anticipation': '#FFC300',
        'trust': '#DAF7A6', 'surprise': '#900C3F', 'sadness': '#3498DB',
        'joy': '#2ECC71', 'disgust': '#6C3483'
    }

    # Anwendung der Farben
    palette = [emotion_colors.get(emotion, '#7F8C8D') for emotion in filtered_data['final_emotion'].unique()]

    # Erstellung des Balkendiagramm
    sns.countplot(
        data=filtered_data,
        x='final_emotion',
        order=filtered_data['final_emotion'].value_counts().index,
        hue='final_emotion',
        palette=palette,
        legend=False
    )
    plt.title(title)
    plt.xlabel('Emotion')
    plt.ylabel('Number of Songs')
    plt.xticks(rotation=45)
    plt.show()

    # Umbenennen der Spalten
    filtered_data = filtered_data.rename(columns={
        'name': 'Songname', 'artists': 'Artist', 'final_emotion': 'Emotion',
        'valence': 'Valence', 'danceability': 'Danceability', 'energy': 'Energy',
        'loudness': 'Loudness', 'tempo': 'Tempo', 'lyrics': 'Lyrics'
    })

    # Anzeige der gefilterten Songs
    if not filtered_data.empty:
        display(filtered_data[['Songname', 'Artist', 'Emotion', 'Valence',
                               'Danceability', 'Energy', 'Loudness', 'Tempo', 'Lyrics']])
    else:
        print("No songs found. Please adjust the filters.")



# Definieren der Dropdown-Menüs und Schieberegler für die verschiedenen Filter

emotion_dropdown = widgets.Dropdown(
    options=['All'] + sorted(data['final_emotion'].unique()),  # Alle Emotionen als Optionen
    value='All',  # Standardwert: Alle Emotionen
    description='Emotion:',
    style={'description_width': 'initial'}
)

# Wenn der Künstler unbekannt ist, setze ihn auf 'Unknown'
data['artists'] = data['artists'].fillna('Unknown').astype(str)
artist_dropdown = widgets.Dropdown(
    options=['All'] + sorted(data['artists'].unique()),  # Alle Künstler als Optionen
    value='All',  # Standardwert: Alle Künstler
    description='Artist:',
    style={'description_width': 'initial'}
)

# Schieberegler für die Spotify-Metriken
valence_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0], min=0.0, max=1.0, step=0.01,
    description='Valence:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

danceability_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0], min=0.0, max=1.0, step=0.01,
    description='Danceability:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

energy_slider = widgets.FloatRangeSlider(
    value=[0.0, 1.0], min=0.0, max=1.0, step=0.01,
    description='Energy:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

loudness_slider = widgets.FloatRangeSlider(
    value=[-60, 0], min=-60, max=0, step=0.5,
    description='Loudness:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

tempo_slider = widgets.FloatRangeSlider(
    value=[50, 200], min=50, max=200, step=1,
    description='Tempo:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

# Textfeld zur Lyrics-Suche
lyrics_search = widgets.Text(
    value='', placeholder='Enter text or words',
    description='Lyrics:', style={'description_width': 'initial'}, layout={'width': '30%'}
)

# Filter-Button, der die Anwendung der Filter auslöst
filter_button = widgets.Button(
    description='Filter', button_style='success', icon='filter'
)

output = widgets.Output()  # Ausgabe-Widget für gefilterte Ergebnisse




# Bereinigung der Lyrics-Sucheingabe
def preprocess_search_text(text):
    """
    Diese Funktion bereinigt den eingegebenen Suchtext:
    - Wandelt den Text in Kleinbuchstaben um
    - Tokenisiert den Text
    - Entfernt Zahlen und Sonderzeichen
    - Entfernt Stopwörter
    """
    tokens = word_tokenize(text.lower())  # Tokenisierung & Kleinschreibung
    tokens = [word for word in tokens if word.isalpha()]  # Sonderzeichen und Zahlen entfernen
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopwörter entfernen
    return tokens



# Filter-Funktion
def apply_filters(change=None):
    """
    Diese Funktion filtert die Daten basierend auf den Benutzereingaben.
    - Filtert nach Emotionen, Künstlern, Spotify-Metriken und Lyrics.
    """
    with output:
        clear_output(wait=True)  # Vorherige Ausgaben löschen
        filtered_data = data.copy()  # Originaldaten kopieren, um Filter anzuwenden

        # Emotionen filtern
        if emotion_dropdown.value != 'All':
            filtered_data = filtered_data[filtered_data['final_emotion'] == emotion_dropdown.value]

        # Künstler filtern
        if artist_dropdown.value != 'All':
            filtered_data = filtered_data[filtered_data['artists'] == artist_dropdown.value]

        # Stimmung filtern
        min_valence, max_valence = valence_slider.value
        filtered_data = filtered_data[
            (filtered_data['valence'] >= min_valence) & (filtered_data['valence'] <= max_valence)
        ]

        # Tanzbarkeit filtern
        min_danceability, max_danceability = danceability_slider.value
        filtered_data = filtered_data[
            (filtered_data['danceability'] >= min_danceability) & (filtered_data['danceability'] <= max_danceability)
        ]

        # Energie filtern
        min_energy, max_energy = energy_slider.value
        filtered_data = filtered_data[
            (filtered_data['energy'] >= min_energy) & (filtered_data['energy'] <= max_energy)
        ]

        # Lautstärke filtern
        min_loudness, max_loudness = loudness_slider.value
        filtered_data = filtered_data[
            (filtered_data['loudness'] >= min_loudness) & (filtered_data['loudness'] <= max_loudness)
        ]

        # Tempo filtern
        min_tempo, max_tempo = tempo_slider.value
        filtered_data = filtered_data[
            (filtered_data['tempo'] >= min_tempo) & (filtered_data['tempo'] <= max_tempo)
        ]

        # Songtext-Suche
        if lyrics_search.value.strip():
            # Bereinigung der Suchworteingabe
            search_tokens = preprocess_search_text(lyrics_search.value.strip())
            print(f"Such-Token: {search_tokens}")  # Anzeige der Tokens, die in den Songtexten gesucht werden
            if search_tokens:
                # Filtern: Alle Tokens müssen in den Songtexten enthalten sein
                filtered_data = filtered_data[
                    filtered_data['clean_lyrics'].apply(lambda tokens: all(token in tokens for token in search_tokens))
                ]
            else:
                print("Die Sucheingabe enthielt keine relevanten Wörter.")

        # Visualisierung der gefilterten Songs
        visualize_emotions_and_display_songs(filtered_data)



# Verknüpfung der Funktion mit dem Filter-Button
filter_button.on_click(apply_filters)



# Layout der Widgets
layout = widgets.VBox([
    widgets.Label("Filter songs based on emotions, artists, Spotify features, and lyrics:"),
    widgets.HBox([emotion_dropdown, artist_dropdown]),
    valence_slider, danceability_slider, energy_slider, loudness_slider, tempo_slider,
    lyrics_search, filter_button, output
])



# Anzeige des Layouts
display(layout)
